<a href="https://colab.research.google.com/github/jingvf/IDS/blob/main/Pytorch_AttentionGRU_DP_experiment%20final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opacus==1.1.3 -q # 指定稳定版本

import opacus
print(opacus.__version__)

1.1.3


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
from opacus import PrivacyEngine
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 加载数据
data_path = '/content/drive/MyDrive/2025paper/dataset/small_dataset.csv'  # 替换为你的数据路径
df = pd.read_csv(data_path)

Using device: cuda


In [3]:
# 输出类别数量
num_classes = df['Class'].nunique()
print(f"总类别数: {num_classes}")

# 输出每个类别的数量
class_counts = df['Class'].value_counts()
print("每个类别的样本数量：")
print(class_counts)
print(df.head())

总类别数: 5
每个类别的样本数量：
Class
Normal          946844
SpoofingRPM      65490
SpoofingGear     59725
DoS              10553
Fuzzy             8967
Name: count, dtype: int64
      Timestamp CAN ID  DLC Data[0] Data[1] Data[2] Data[3] Data[4] Data[5]  \
0  1.478192e+09    316    8      45      29      24      ff      29      24   
1  1.478192e+09    316    8      45      29      24      ff      29      24   
2  1.478195e+09   0140    8      00      00      00      00      08      28   
3  1.478191e+09   0545    8      d8      00      00      8a      00      00   
4  1.478195e+09   043f    8       1      45      60      ff      6b       0   

  Data[6] Data[7]         Class  
0       0      ff   SpoofingRPM  
1       0      ff   SpoofingRPM  
2      2f      15        Normal  
3      00      00        Normal  
4       0       0  SpoofingGear  


In [4]:
# 1. CAN ID 转成 int（十六进制字符串 → 十进制）
# =====================
df["CAN ID"] = df["CAN ID"].apply(lambda x: int(str(x), 16) if isinstance(x, str) else int(x))

# 特征列
feature_cols = ["CAN ID", "DLC"] + [f"Data[{i}]" for i in range(8)]
X_features = df[feature_cols].astype(str).applymap(lambda x: int(x, 16) if isinstance(x, str) else int(x)).values

# 标签
y = df["Class"].values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# 数据维度调整 (B, T, F)，这里把 T=1 表示每条 CAN 消息为一个时间步
X_scaled = X_scaled[:, np.newaxis, :]  # shape -> [num_samples, 1, num_features]

# =====================
# 5. 类别权重 (用统计的数量，如果需要自动算就用 np.bincount(y_encoded))
# =====================
counts = [946844, 65490, 59725, 10553, 8967]  # 示例：之前统计好的数量
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


/tmp/ipython-input-1351706014.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_features = df[feature_cols].astype(str).applymap(lambda x: int(x, 16) if isinstance(x, str) else int(x)).values


Using device: cuda


In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 转 Tensor
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                            torch.tensor(y_val, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(y_test, dtype=torch.long))

BATCH_SIZE = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# 3. 模型定义：CNN-Attention (DP友好)
# ================================
class DPMultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.shape
        Q = self.q_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.k_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        attn_scores = (Q @ K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_out = attn_weights @ V
        attn_out = attn_out.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(attn_out)

class DPCNNAttention(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=4, hidden_dim=128):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.gn1 = nn.GroupNorm(num_groups=8, num_channels=hidden_dim)
        self.gn2 = nn.GroupNorm(num_groups=8, num_channels=hidden_dim)
        self.attn = DPMultiHeadSelfAttention(hidden_dim, num_heads)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = F.relu(self.gn1(self.conv1(x)))
        x = F.relu(self.gn2(self.conv2(x)))
        x = x.transpose(1, 2)
        attn_out = self.attn(x)
        pooled = attn_out.mean(dim=1)
        return self.fc(pooled)

In [7]:
model = DPCNNAttention(11, num_classes).to(device)

from torchsummary import summary
summary(model, input_size=(100, 11))  # 序列长度为10

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 128, 100]           4,352
         GroupNorm-2             [-1, 128, 100]             256
            Conv1d-3             [-1, 128, 100]          49,280
         GroupNorm-4             [-1, 128, 100]             256
            Linear-5             [-1, 100, 128]          16,512
            Linear-6             [-1, 100, 128]          16,512
            Linear-7             [-1, 100, 128]          16,512
            Linear-8             [-1, 100, 128]          16,512
DPMultiHeadSelfAttention-9             [-1, 100, 128]               0
           Linear-10                    [-1, 5]             645
Total params: 120,837
Trainable params: 120,837
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.88
Params size (MB): 0.46
Estim

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from opacus import PrivacyEngine
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def run_dp_comparison(train_loader, val_loader, test_loader,
                      input_dim, num_classes, target_epsilons=[1,3,5],
                      epochs=20, device="cuda"):

    results = {"SGD": {}, "Adam": {}}

    for eps in target_epsilons:
        for opt_name in ["SGD", "Adam"]:
            print(f"\nRunning {opt_name} with target ε={eps}")

            model = DPCNNAttention(input_dim=input_dim, num_classes=num_classes).to(device)
            optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) if opt_name=="SGD" else torch.optim.Adam(model.parameters(), lr=1e-3)
            criterion = nn.CrossEntropyLoss()

            privacy_engine = PrivacyEngine()
            model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
                module=model,
                optimizer=optimizer,
                data_loader=train_loader,
                epochs=epochs,
                target_epsilon=eps,
                target_delta=1e-5,
                max_grad_norm=1.0,
            )

            train_losses, val_accs, epsilons_list = [], [], []

            for epoch in range(epochs):
                model.train()
                total_loss, total_correct, total_samples = 0, 0, 0
                for x, y in train_loader:
                    x, y = x.to(device), y.to(device)
                    optimizer.zero_grad()
                    out = model(x)
                    loss = criterion(out, y)
                    loss.backward()
                    optimizer.step()

                    total_loss += loss.item() * y.size(0)
                    total_correct += (out.argmax(1)==y).sum().item()
                    total_samples += y.size(0)
                train_losses.append(total_loss/total_samples)

                # Val
                model.eval()
                val_correct, val_total = 0,0
                with torch.no_grad():
                    for x, y in val_loader:
                        x, y = x.to(device), y.to(device)
                        out = model(x)
                        val_correct += (out.argmax(1)==y).sum().item()
                        val_total += y.size(0)
                val_acc = val_correct / val_total
                val_accs.append(val_acc)

                eps_curr = privacy_engine.accountant.get_epsilon(delta=1e-5)
                epsilons_list.append(eps_curr)

                print(f"[{opt_name}] Epoch {epoch+1}/{epochs}: Loss={train_losses[-1]:.4f}, Val Acc={val_acc:.4f}, ε={eps_curr:.2f}")

            # Test
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for x, y in test_loader:
                    x, y = x.to(device), y.to(device)
                    out = model(x)
                    y_true.extend(y.cpu().numpy())
                    y_pred.extend(out.argmax(1).cpu().numpy())
            conf_mat = confusion_matrix(y_true, y_pred)

            results[opt_name][eps] = {
                "train_losses": train_losses,
                "val_accs": val_accs,
                "epsilons": epsilons_list,
                "conf_mat": conf_mat,
                "model": model  # 保存最终模型
            }

    return results

In [15]:
# 1️⃣ 准备 DataLoader（train_loader, val_loader, test_loader）和参数
input_dim = X_scaled.shape[2]  # 特征维度
num_classes = len(set(y_encoded))
target_epsilons = [1, 3, 5, 7]  # 你想对比的 epsilon 值
epochs = 10

# 2️⃣ 运行 DP-SGD / DP-Adam 对比实验，得到 results
results = run_dp_comparison(train_loader, val_loader, test_loader,
                             input_dim=input_dim,
                             num_classes=num_classes,
                             target_epsilons=target_epsilons,
                             epochs=epochs,
                             device=device)


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(



Running SGD with target ε=1


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


[SGD] Epoch 1/10: Loss=0.6813, Val Acc=0.8678, ε=0.90
[SGD] Epoch 2/10: Loss=0.4680, Val Acc=0.8678, ε=0.92
[SGD] Epoch 3/10: Loss=0.3125, Val Acc=0.9278, ε=0.93
[SGD] Epoch 4/10: Loss=0.2321, Val Acc=0.9782, ε=0.94
[SGD] Epoch 5/10: Loss=0.1845, Val Acc=0.9762, ε=0.95
[SGD] Epoch 6/10: Loss=0.1659, Val Acc=0.9749, ε=0.96
[SGD] Epoch 7/10: Loss=0.1632, Val Acc=0.9751, ε=0.97
[SGD] Epoch 8/10: Loss=0.1577, Val Acc=0.9756, ε=0.98
[SGD] Epoch 9/10: Loss=0.1539, Val Acc=0.9762, ε=0.99
[SGD] Epoch 10/10: Loss=0.1479, Val Acc=0.9765, ε=0.99

Running Adam with target ε=1


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


...

[Adam] Epoch 1/10: Loss=0.0482, Val Acc=0.9967, ε=3.77
[Adam] Epoch 2/10: Loss=0.0148, Val Acc=0.9977, ε=4.02
[Adam] Epoch 3/10: Loss=0.0146, Val Acc=0.9978, ε=4.21
[Adam] Epoch 4/10: Loss=0.0142, Val Acc=0.9980, ε=4.35
[Adam] Epoch 5/10: Loss=0.0143, Val Acc=0.9979, ε=4.49
[Adam] Epoch 6/10: Loss=0.0136, Val Acc=0.9975, ε=4.60
[Adam] Epoch 7/10: Loss=0.0136, Val Acc=0.9980, ε=4.71
[Adam] Epoch 8/10: Loss=0.0145, Val Acc=0.9984, ε=4.82
[Adam] Epoch 9/10: Loss=0.0151, Val Acc=0.9985, ε=4.91
[Adam] Epoch 10/10: Loss=0.0146, Val Acc=0.9987, ε=5.00

Running SGD with target ε=7


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


[SGD] Epoch 1/10: Loss=0.6757, Val Acc=0.8678, ε=4.95
[SGD] Epoch 2/10: Loss=0.4756, Val Acc=0.8678, ε=5.36
[SGD] Epoch 3/10: Loss=0.3165, Val Acc=0.9278, ε=5.65
[SGD] Epoch 4/10: Loss=0.2298, Val Acc=0.9756, ε=5.91
[SGD] Epoch 5/10: Loss=0.1864, Val Acc=0.9741, ε=6.12
[SGD] Epoch 6/10: Loss=0.1711, Val Acc=0.9732, ε=6.33
[SGD] Epoch 7/10: Loss=0.1695, Val Acc=0.9735, ε=6.51
[SGD] Epoch 8/10: Loss=0.1662, Val Acc=0.9741, ε=6.67
[SGD] Epoch 9/10: Loss=0.1635, Val Acc=0.9746, ε=6.83
[SGD] Epoch 10/10: Loss=0.1590, Val Acc=0.9751, ε=6.99

Running Adam with target ε=7


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


[Adam] Epoch 1/10: Loss=0.0479, Val Acc=0.9972, ε=4.95
[Adam] Epoch 2/10: Loss=0.0162, Val Acc=0.9971, ε=5.36
[Adam] Epoch 3/10: Loss=0.0148, Val Acc=0.9978, ε=5.65
[Adam] Epoch 4/10: Loss=0.0129, Val Acc=0.9986, ε=5.91
[Adam] Epoch 5/10: Loss=0.0116, Val Acc=0.9981, ε=6.12
[Adam] Epoch 6/10: Loss=0.0119, Val Acc=0.9976, ε=6.33
[Adam] Epoch 7/10: Loss=0.0126, Val Acc=0.9983, ε=6.51
[Adam] Epoch 8/10: Loss=0.0125, Val Acc=0.9986, ε=6.67
[Adam] Epoch 9/10: Loss=0.0123, Val Acc=0.9984, ε=6.83
[Adam] Epoch 10/10: Loss=0.0125, Val Acc=0.9984, ε=6.99


In [1]:
# 1️⃣ 可视化函数
# ================================
def plot_epochs_curves(results, epsilon):
    optimizers = ["SGD", "Adam"]
    plt.figure(figsize=(12,5))

    # Loss
    plt.subplot(1,2,1)
    for opt in optimizers:
        res = results[opt][epsilon]
        epochs = range(1, len(res["train_losses"])+1)
        plt.plot(epochs, res["train_losses"], label=f"{opt} Train Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.title(f"Training Loss per Epoch (ε={epsilon})")
    plt.legend()

    # Val Accuracy
    plt.subplot(1,2,2)
    for opt in optimizers:
        res = results[opt][epsilon]
        epochs = range(1, len(res["val_accs"])+1)
        plt.plot(epochs, res["val_accs"], label=f"{opt} Val Acc")
    plt.xlabel("Epoch"); plt.ylabel("Validation Accuracy")
    plt.title(f"Validation Accuracy per Epoch (ε={epsilon})")
    plt.legend()

    plt.show()

def plot_epsilon_comparison(results, metric="val_accs"):
    optimizers = ["SGD", "Adam"]
    epsilons = sorted(list(results["SGD"].keys()))

    plt.figure(figsize=(12,5))

    for opt in optimizers:
        metric_vals = []
        for eps in epsilons:
            res = results[opt][eps]
            if metric=="epsilons":
                metric_vals.append(res["epsilons"][-1])
            else:
                metric_vals.append(res[metric][-1])
        plt.plot(epsilons, metric_vals, marker='o', label=f"{opt}")

    plt.xlabel("Target ε")
    ylabel = "Value"
    if metric=="train_losses":
        ylabel = "Train Loss"
    elif metric=="val_accs":
        ylabel = "Validation Accuracy"
    elif metric=="epsilons":
        ylabel = "Achieved Epsilon"
    plt.ylabel(ylabel)
    plt.title(f"{ylabel} vs Target Epsilon")
    plt.legend()
    plt.show()

def plot_confusion_matrix(conf_mat, class_names, title="Confusion Matrix"):
    plt.figure(figsize=(6,5))
    sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.title(title)
    plt.show()

In [ ]:

# 3️⃣ 可视化某个 epsilon 下的训练曲线
plot_epochs_curves(results, epsilon=5)

# 4️⃣ 可视化不同 epsilon 下最终 Val Acc / Loss / Achieved ε
plot_epsilon_comparison(results, metric="val_accs")
plot_epsilon_comparison(results, metric="train_losses")
plot_epsilon_comparison(results, metric="epsilons")

# 5️⃣ 混淆矩阵
class_names = le.classes_  # 你的类别名
plot_confusion_matrix(results["SGD"][5]["conf_mat"], class_names, title="DP-SGD Confusion Matrix")
plot_confusion_matrix(results["Adam"][5]["conf_mat"], class_names, title="DP-Adam Confusion Matrix")
